In het PowerBI report van sewer ontbreekt nu een kolom met de originele meetwaarden. Deze is nodig voor één van de weekrapporten. Aan de tabel VWSDEST.Sewer\_Prolongated wordt daarvoor een kolom toegevoegd die deze originele waarde bevat. Vervolgens wordt de nieuwe kolom gevuld in de Stored Procedure sp\_Sewer\_Prolongated en toegevoegd in de view VWSREPORT.V\_PBI\_Riool\_Dagwaarden.

# **DEPENDENCIES**
---

```json
{
    "depends-on": [
        "src/dataflows/rivm_sewer_measurements.ipynb"
    ]
}
```

**ALTER TABLE**

In [ ]:

IF NOT EXISTS 
    (SELECT * FROM SYS.COLUMNS WHERE [NAME] IN (N'RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL') AND [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[SEWER_PROLONGATED]'))
	BEGIN
        ALTER TABLE VWSDEST.SEWER_PROLONGATED ADD [RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL] FLOAT;
    END
GO

**ALTER PROCEDURE**

In [ ]:
-- ALTER STORED PROCEDURE
 CREATE OR ALTER  PROCEDURE [dbo].[SP_SEWER_PROLONGATED]
 AS
 BEGIN
 WITH BASE_CTE AS (
 
     SELECT 
         WEEK
         ,DATE_MEASUREMENT
         ,RWZI_AWZI_CODE 
         ,RWZI_AWZI_NAME
         ,RNA_FLOW_PER_100000 
         ,RNA_FLOW_PER_100000 / 1.0E+11 AS RNA_FLOW_PER_100000_PER_RWZI
     FROM VWSDEST.SEWER_BASE
     WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSDEST.SEWER_BASE)
         -- KEEP ONLY THE FIRST MEASUREMENT (SOME DUPLICATE MEASUREMENTS PER DAY PER RWZI ARE PRESENT)
         AND COUNT_DAILY_MEASUREMENT=1
 )
 ,
 RWZI_CTE AS (
     -- ALLOCATION TABLE WITH DATA ON GM AND VR LEVEL
     SELECT 
         [RWZI_CODE],
         [RWZI_NAAM],
         [STARTDATUM],
         [EINDDATUM],
         [INWONERS],
         [REGIO_TYPE],
         [REGIO_CODE],
         [REGIO_NAAM],
         [AANDEEL],
         [INWONERS] * ([AANDEEL]/100) AS [COVERED_INHABITANTS]
     FROM VWSSTATIC.CBS_POPULATION_RWZI
     WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_RWZI)
         AND REGIO_TYPE = 'VR' -- GM WILL NOT BE USING THIS METHOD.
     
     UNION ALL
 
         SELECT DISTINCT
         [RWZI_CODE],
         [RWZI_NAAM],
         [STARTDATUM],
         [EINDDATUM],
         [INWONERS],
         'NL'            AS [REGIO_TYPE],
         'NL'            AS [REGIO_CODE],
         'Netherlands'   AS [REGIO_NAAM],
         100.00          AS [AANDEEL],
         [INWONERS]      AS [COVERED_INHABITANTS]
     FROM VWSSTATIC.CBS_POPULATION_RWZI
     WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTATIC.CBS_POPULATION_RWZI)
         AND REGIO_TYPE = 'VR' -- CHOOSE EITHER VR OR GM FOR RARE CASES WHERE INHABITANTS DO NOT ALIGN SUCH AS RWZI 9041
 
 ),
 GRID_CTE AS (
     -- USE CALENDAR TABLE TO CREATE A RECORD FOR EACH POSSIBLE COMBINATION OF DATE, RWZI AND REGION
 SELECT  
          T0.DATE_MEASUREMENT
         ,T1.RWZI_CODE
         ,T1.[REGIO_CODE]
         ,T1.[REGIO_NAAM]
     FROM 
         (
             SELECT CAST([DAY] AS DATETIME) AS  [DATE_MEASUREMENT] FROM [VWSSTATIC].[CALENDAR]
                 WHERE   [DAY]>= (SELECT MIN(DATE_MEASUREMENT) FROM BASE_CTE)
                 AND [DAY]<= (SELECT MAX(DATE_MEASUREMENT) FROM BASE_CTE)
         ) AS T0
     CROSS JOIN 
         (SELECT DISTINCT 
              [RWZI_CODE]
             ,[REGIO_CODE]
             ,[REGIO_NAAM]
         FROM RWZI_CTE) AS T1
     --ADD ADDITIONAL (DE-DUPLICATED) INFO TO RWZI AND REGIO DIMENSIONS
 ),
 DAILY_CTE AS (
     SELECT 
          T0.WEEK
         ,T0.DATE_MEASUREMENT
         ,T0.RWZI_AWZI_CODE
         ,T0.RWZI_AWZI_NAME
         -- ,T0.RNA_FLOW_PER_100000 
         ,T0.RNA_FLOW_PER_100000_PER_RWZI
         ,T1.STARTDATUM
         ,T1.EINDDATUM
         ,T1.REGIO_CODE
         ,T1.AANDEEL
         ,T1.INWONERS
         ,T1.COVERED_INHABITANTS
     FROM BASE_CTE AS T0
     LEFT JOIN RWZI_CTE AS T1
         ON  T0.RWZI_AWZI_CODE = T1.RWZI_CODE
         AND T1.STARTDATUM <= T0.DATE_MEASUREMENT AND T1.EINDDATUM > T0.DATE_MEASUREMENT
 ),
 SPARSE_CTE AS
 (
     SELECT 
          T0.DATE_MEASUREMENT
         ,T0.RWZI_CODE
         ,T0.[REGIO_CODE]
         ,T0.[REGIO_NAAM]
         ,'|' AS [|]
         ,T1.WEEK
         ,T1.RWZI_AWZI_NAME
         ,T1.RNA_FLOW_PER_100000_PER_RWZI AS RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL
         ,T1.AANDEEL
         ,T1.INWONERS
         ,T1.COVERED_INHABITANTS
         --DETERMINE THE LAST DATE THAT CONTAINED VALUES FOR THIS RWZI AND REGION
         ,MAX(CASE WHEN T1.REGIO_CODE IS NOT NULL THEN T0.[DATE_MEASUREMENT] END)
             OVER (PARTITION BY T0.RWZI_CODE, T0.REGIO_CODE ORDER BY T0.[DATE_MEASUREMENT] ROWS 8 PRECEDING) AS PROLONGATE_DATE
     FROM GRID_CTE AS T0
     LEFT JOIN DAILY_CTE AS T1 ON T0.DATE_MEASUREMENT = T1.DATE_MEASUREMENT AND T0.RWZI_CODE = T1.RWZI_AWZI_CODE AND T0.REGIO_CODE = T1.REGIO_CODE
 ),
 DENSE_CTE AS (
 SELECT 
      *
      --ADD THE TOTAL NUMBER OF INHABITANTS IN THE REGION BASED ON THE PROLONGATED INHABITANT NUMBERS
     ,SUM(COVERED_INHABITANTS)                   OVER (PARTITION BY            REGIO_CODE, DATE_MEASUREMENT) AS REGION_COVERED_INHABITANTS
     FROM (
         --PROLONGATE THE VALUES: RETRIEVE THE ONLY VALUE DETERMINED FOR PROLONGATE DATE, RWZI AND REGION COMBINATION
         SELECT 
              DATE_MEASUREMENT
             ,RWZI_CODE
             ,[REGIO_CODE]
             ,[REGIO_NAAM]
             ,[|]
             ,MAX(RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL) OVER (PARTITION BY RWZI_CODE, REGIO_CODE, PROLONGATE_DATE) AS RNA_FLOW_PER_100000_PER_RWZI
             ,MAX(COVERED_INHABITANTS) OVER (PARTITION BY RWZI_CODE, REGIO_CODE, PROLONGATE_DATE) AS COVERED_INHABITANTS
             ,AANDEEL
             ,INWONERS
             -- ,COVERED_INHABITANTS
             ,PROLONGATE_DATE
             , RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL
         FROM SPARSE_CTE
         ) T0   
 )
     INSERT INTO VWSDEST.SEWER_PROLONGATED(
          [DATE_MEASUREMENT]                     
         ,[REGIO_CODE]                           
         ,[WEEK]                                 
         ,[WEEK_UNIX]                            
         ,[AVERAGE_RNA_FLOW_PER_100000]
         ,[RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL]          
         ,[REGION_COVERED_INHABITANTS]      
     )
     SELECT 
         [DATE_MEASUREMENT]
         ,[REGIO_CODE]
         ,DATEPART(ISO_WEEK, [DATE_MEASUREMENT]) AS WEEK
         ,[DBO].[CONVERT_DATETIME_TO_UNIX](DATEADD(WEEK, DATEDIFF(WEEK, 0, DATE_MEASUREMENT - 1), 0)) AS WEEK_UNIX
         ,CASE  
             WHEN [REGION_COVERED_INHABITANTS] != 0 THEN SUM( (COVERED_INHABITANTS * RNA_FLOW_PER_100000_PER_RWZI) ) / REGION_COVERED_INHABITANTS
             ELSE  SUM( ([COVERED_INHABITANTS] * RNA_FLOW_PER_100000_PER_RWZI) ) /1 
         END 
         AS [AVERAGE_RNA_FLOW_PER_100000]
         , RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL
         ,[REGION_COVERED_INHABITANTS]
     FROM DENSE_CTE
     GROUP BY DATE_MEASUREMENT, [REGIO_CODE], REGION_COVERED_INHABITANTS;
 END
 
GO;


**ALTER VIEW**

In [ ]:
CREATE OR ALTER VIEW [VWSREPORT].[V_PBI_Riool_Dagwaarden]
AS
-- Riool gegevens      
    SELECT
        CAST([DATE_MEASUREMENT] as date)                                        AS [Datum]
        ,CAST(dbo.CONVERT_UNIX_TO_DATETIME([WEEK_UNIX]) as date)	            AS [Week start]
        ,CAST(dbo.WEEK_END(dbo.CONVERT_UNIX_TO_DATETIME([WEEK_UNIX])) as date)  AS [Week einde]
        ,CAST([DATE_LAST_INSERTED] as date)                                     AS [Update datum]
        ,[AVERAGE_RNA_FLOW_PER_100000]							                AS [RNA flow per 100K]
        ,[REGION_COVERED_INHABITANTS]								            AS [Inwoners regio covered]
        ,[RNA_FLOW_PER_100000_PER_RWZI_ORIGINAL]                                AS [RNA flow per 100K dagwaarde]
  FROM [VWSDEST].[SEWER_PROLONGATED]
  WHERE
	[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSDEST].[SEWER_PROLONGATED])
	and
	REGIO_CODE = 'NL'
GO